In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import EarlyStopping
from sklearn.cross_validation import train_test_split
import numpy as np
import time
import json
import time
import datetime
from keras.models import load_model
import sys, tensorflow, keras
from keras import backend as K

C:\Users\jmo4cob\AppData\Local\conda\conda\envs\tf35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\jmo4cob\AppData\Local\conda\conda\envs\tf35\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


#### Version

In [2]:
sys.version

'3.5.5 |Anaconda, Inc.| (default, Mar 12 2018, 17:44:09) [MSC v.1900 64 bit (AMD64)]'

In [3]:
tensorflow.__version__

'1.4.0'

In [4]:
keras.__version__

'2.0.8'

In [5]:
conv_arch=[(32,3),(64,3),(128,3)]
dense=[64,2]
dropout=0.5
batch_size=128
nb_epoch=100
validation_split=0.2
patience=5
dirpath='model/'

In [6]:
def describe(X_shape, y_shape, batch_size, dropout, nb_epoch, conv_arch, dense):
    print (' X_train shape: ', X_shape) # (n_sample, 1, 48, 48)
    print (' y_train shape: ', y_shape) # (n_sample, n_categories)
    print ('      img size: ', X_shape[1], X_shape[2])
    print ('    batch size: ', batch_size)
    print ('      nb_epoch: ', nb_epoch)
    print ('       dropout: ', dropout)
#     print ('conv architect: ', conv_arch)
#     print ('neural network: ', dense)

In [7]:
# import dataset:
X_fname = 'dataset/X_train_data_Training.npy'
y_fname = 'dataset/y_train_data_Training.npy'
X_train = np.load(X_fname)
y_train = np.load(y_fname)


In [8]:
X_val = np.load('dataset/X_train_data_PublicTest.npy')
y_val = np.load('dataset/y_train_data_PublicTest.npy')
X_val = X_val.astype('float32')

In [9]:
starttime = time.time()
X_train = X_train.astype('float32')
X_shape = X_train.shape
y_shape = y_train.shape
describe(X_shape, y_shape, batch_size, dropout, nb_epoch, conv_arch, dense)

 X_train shape:  (28709, 48, 48, 1)
 y_train shape:  (28709, 6)
      img size:  48 48
    batch size:  128
      nb_epoch:  100
       dropout:  0.5


## Build Model

In [10]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.models import Model

In [11]:
from keras.layers import Input

In [12]:
InceptionV3_model = InceptionV3(weights='imagenet', include_top=False)

In [13]:
VGG16_model = VGG16(weights='imagenet', include_top=False)

In [14]:
X_train[:1].shape

(1, 48, 48, 1)

In [15]:
K.image_data_format()

'channels_last'

#### VGG16 Model Summary

In [16]:
VGG16_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [17]:
VGG16_model_1 = Model(inputs=VGG16_model.get_input_at(0), outputs=VGG16_model.get_layer('block4_pool').output)
VGG16_model_2 = Model(inputs=VGG16_model.get_input_at(0), outputs=VGG16_model.get_layer('block5_pool').output)

In [18]:
#VGG16_model_2.summary()

In [19]:
input_tensor = Input(shape=(139, 139, 3))
InceptionV3_model = InceptionV3(input_tensor=input_tensor, input_shape=(139, 139, 3), weights='imagenet', include_top=False)

In [20]:
input_tensor = Input(shape=(48, 48, 3))
VGG16_model = VGG16(input_tensor=input_tensor,  weights='imagenet', include_top=False)

#### Base Model

In [21]:
base_model = Sequential()
# model.add(in_model)
# model.add(BatchNormalization())
base_model.add(Conv2D(3, kernel_size=1, strides=(1,1), input_shape=(48, 48, 1) , padding = 'same', 
                  activation = 'relu',data_format='channels_last'))
#model.add(Conv2D(3, kernel_size = (1,1), padding = 'same', activation = 'relu'))
#base_model.add(VGG16_model)

In [22]:
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_189 (Conv2D)          (None, 48, 48, 3)         6         
Total params: 6
Trainable params: 6
Non-trainable params: 0
_________________________________________________________________


In [23]:
y_train.shape[1]

6

#### Add VGG16 Model to Base

In [24]:
# add a global spatial average pooling layer
x = base_model.output
x = VGG16_model(x)
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(y_train.shape[1], activation='softmax')(x)

In [25]:
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

In [26]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False
for layer in model.layers:
    if layer.name == 'vgg16':
        layer.trainable = False

In [27]:
for i, layer in enumerate(model.layers):
    print(i, layer.name, '---' ,layer.trainable)

0 conv2d_189_input --- False
1 conv2d_189 --- False
2 vgg16 --- False
3 global_average_pooling2d_1 --- True
4 dense_1 --- True
5 dense_2 --- True


In [28]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [29]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_189_input (InputLayer (None, 48, 48, 1)         0         
_________________________________________________________________
conv2d_189 (Conv2D)          (None, 48, 48, 3)         6         
_________________________________________________________________
vgg16 (Model)                (None, 1, 1, 512)         14714688  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              525312    
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 6150      
Total params: 15,246,156
Trainable params: 531,462
Non-trainable params: 14,714,694
__________________________________________________________

In [30]:
model.fit(x=X_train, y=y_train, batch_size=128, epochs=5, verbose=1, callbacks=None, 
    validation_split=0.0, validation_data=(X_val,y_val), shuffle=True, class_weight=None, 
    sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None)

Train on 28709 samples, validate on 3589 samples
Epoch 1/5
28709/28709 [==============================] - 50s - loss: 11.2739 - val_loss: 10.4094
Epoch 2/5
28709/28709 [==============================] - 47s - loss: 6.4074 - val_loss: 1.7714
Epoch 3/5
28709/28709 [==============================] - 47s - loss: 1.5941 - val_loss: 1.6857
Epoch 4/5
28709/28709 [==============================] - 47s - loss: 1.4069 - val_loss: 1.6493
Epoch 5/5
28709/28709 [==============================] - 47s - loss: 1.2767 - val_loss: 1.6991s: 1.2 - ETA: 2s -


In [31]:
model.save('model/Transfer_learning_model-1.hdf5')

### Change the Trainable Layers and Retrain

In [32]:
for layer in base_model.layers:
    layer.trainable = True

In [33]:
for i, layer in enumerate(model.layers):
    print(i, layer.name,'---' ,layer.trainable)


0 conv2d_189_input --- False
1 conv2d_189 --- True
2 vgg16 --- False
3 global_average_pooling2d_1 --- True
4 dense_1 --- True
5 dense_2 --- True


In [34]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [35]:
model.fit(x=X_train, y=y_train, batch_size=128, epochs=5, verbose=1, callbacks=None, 
    validation_split=0.0, validation_data=(X_val,y_val), shuffle=True, class_weight=None, 
    sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None)

Train on 28709 samples, validate on 3589 samples
Epoch 1/5
28709/28709 [==============================] - 96s - loss: 1.1720 - val_loss: 1.8359
Epoch 2/5
28709/28709 [==============================] - 94s - loss: 1.0357 - val_loss: 1.8773
Epoch 3/5
28709/28709 [==============================] - 93s - loss: 0.9279 - val_loss: 2.0047
Epoch 4/5
28709/28709 [==============================] - 94s - loss: 0.8298 - val_loss: 2.1234
Epoch 5/5
28709/28709 [==============================] - 94s - loss: 0.7425 - val_loss: 2.4052


In [36]:
model.save('model/Transfer_learning_model-2.hdf5')

### Change the Trainable Layers and Retrain - Train all layers

In [37]:
for layer in model.layers:
    layer.trainable = True
for i, layer in enumerate(model.layers):
    print(i, layer.name,'---' ,layer.trainable)

0 conv2d_189_input --- True
1 conv2d_189 --- True
2 vgg16 --- True
3 global_average_pooling2d_1 --- True
4 dense_1 --- True
5 dense_2 --- True


In [38]:
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

In [39]:
model.fit(x=X_train, y=y_train, batch_size=128, epochs=10, verbose=1, callbacks=None, 
    validation_split=0.0, validation_data=(X_val,y_val), shuffle=True, class_weight=None, 
    sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None)

Train on 28709 samples, validate on 3589 samples
Epoch 1/10
28709/28709 [==============================] - 148s - loss: 0.8529 - val_loss: 1.9648
Epoch 2/10
28709/28709 [==============================] - 146s - loss: 0.6670 - val_loss: 2.0588
Epoch 3/10
28709/28709 [==============================] - 147s - loss: 0.5683 - val_loss: 2.1536
Epoch 4/10
28709/28709 [==============================] - 146s - loss: 0.4854 - val_loss: 2.1810
Epoch 5/10
28709/28709 [==============================] - 147s - loss: 0.4245 - val_loss: 2.2715
Epoch 6/10
28709/28709 [==============================] - 146s - loss: 0.3695 - val_loss: 2.3141
Epoch 7/10
28709/28709 [==============================] - 147s - loss: 0.3192 - val_loss: 2.4810
Epoch 8/10
28709/28709 [==============================] - 146s - loss: 0.2884 - val_loss: 2.5783
Epoch 9/10
28709/28709 [==============================] - 146s - loss: 0.2607 - val_loss: 2.7147
Epoch 10/10
28709/28709 [==============================] - 146s - loss: 0.2365

In [40]:
model.save('model/Transfer_learning_model-3.hdf5')